In [1]:
import os
import pandas as pd
import glob

In [2]:
path_input=os.path.join(os.getcwd(), '../data/input/dresden/2019/')
path_output_weather = os.path.join(os.getcwd(), '../data/output/weather/')
path_output=os.path.join(os.getcwd(), '../data/output/')

## Read Data

In [3]:
directors = glob.glob(path_input+"*")

all_files = []
for director in directors:
    all_files=all_files+glob.glob(director+"/*.csv")
    
li = []

for filename in all_files:
    df_read = pd.read_csv(filename, index_col=None, header=0)
    li.append(df_read)

df = pd.concat(li, axis=0, ignore_index=True)

## Drop Columns

In [4]:
#Columns including single value
#The following columns only included one value (or NaN values)
#city : ['dresden']
#p_rack_locks : [False]
#b_state : ['ok']
#b_active : [ True]
#b_battery_pack : [nan '{"percentage": 0}']
#p_special_racks : [nan 0.]
#p_free_special_racks : [nan 0.]

#Columns which highly correlate

#The following columns highly correlate - meaning they are complementary booleans or if one is high, the other is too

#p_bike [corr(p_spot)=-0.999501, complementary boolean to p_spot]

df_cleaned = df.drop(["city", 
                      "p_bike", 
                      "p_rack_locks", 
                      "b_active", 
                      "b_state", 
                      "b_battery_pack", 
                      "p_rack_locks", 
                      "p_special_racks", 
                      "p_free_special_racks"], axis=1)
df_cleaned.head()

,p_spot,b_lock_types,p_maintenance,p_bike_racks,p_terminal_type,p_place_type,p_number,p_uid,b_number,p_free_racks,...,p_address,b_electric_lock,p_bikes,trip,p_booked_bikes,p_name,b_bike_type,p_lat,b_pedelec_battery,p_bike_types
0,True,frame_lock,False,0,free,0,4505.0,10299865,93153,0,...,NaN,True,3,first,0,Altleubitz,0,51.015597,0.0,"{""undefined"": 2, ""15"": 1}"
1,True,frame_lock,False,0,free,0,4505.0,10299865,93153,0,...,NaN,True,3,last,0,Altleubitz,0,51.015597,0.0,"{""undefined"": 2, ""15"": 1}"
2,False,frame_lock,False,0,NaN,12,0.0,13101449,93616,0,...,NaN,True,1,first,0,BIKE 93616,15,51.065062,0.0,"{""15"": 1}"
3,False,frame_lock,False,0,NaN,12,0.0,13101449,93616,0,...,NaN,True,1,start,0,BIKE 93616,15,51.065062,0.0,"{""15"": 1}"
4,False,frame_lock,False,0,NaN,12,0.0,13103279,93616,0,...,NaN,True,1,end,0,BIKE 93616,15,51.064738,0.0,"{""15"": 1}"


## Create Trips

In [5]:
# No Duplicates
len(df_cleaned)-len(df_cleaned.drop_duplicates(subset=df_cleaned.columns.difference(["p_lat", "p_lng"])))

0

In [6]:
print("Starts:", len(df_cleaned[df_cleaned["trip"]=="start"]))
print("Ends:", len(df_cleaned[df_cleaned["trip"]=="end"]))
print("Difference:", len(df_cleaned[df_cleaned["trip"]=="start"])-len(df_cleaned[df_cleaned["trip"]=="end"]))

Starts: 565996
Ends: 550350
Difference: 15646


In [7]:
# df_2: start and end of trips
df_2 = df_cleaned[(df_cleaned["trip"]=="start") | (df_cleaned["trip"]=="end")]
print("Dropped", len(df_cleaned)-len(df_2), "first and last Values")

Dropped 290120 first and last Values


In [8]:
# df_3: start and end of trips sorted by b_number and datetime
df_3=df_2.sort_values(["b_number", "datetime"])
df_3[["b_number", "datetime", "trip"]].iloc[500010:500030] # two examples for multiple starts

,b_number,datetime,trip
702798,93554,2019-07-05 13:37:00,start
702799,93554,2019-07-05 13:46:00,end
702800,93554,2019-07-05 14:20:00,start
702801,93554,2019-07-05 14:35:00,end
702802,93554,2019-07-05 16:44:00,start
702803,93554,2019-07-05 16:51:00,start
702804,93554,2019-07-05 17:29:00,end
702805,93554,2019-07-05 18:48:00,start
702806,93554,2019-07-05 19:18:00,end
702807,93554,2019-07-05 20:20:00,start


In [9]:
# sr_1: True if after start entry comes end entry else False
sr_1 = (df_3['trip'] != df_3['trip'].shift())
df_3["valid_start"]=sr_1

# sr_2: 
#sr_2 = (df_3['trip'] != df_3['trip'].shift(-1))
#df_3['valid_end'] = sr_2


In [10]:
# df_4: start and end of trips sorted by b_number and datetime only valid starts
df_4=df_3[df_3["valid_start"]==True]
print("Deleted entries", len(df_3)-len(df_4))
print("Correct entries", len(df_4))

Deleted entries 15646
Correct entries 1100700


In [11]:
# Tests if there are values with two ends after each other
test = df_4["trip"] != df_4["trip"].shift(-1)
print("Entries where Bike X ends with an end and bike Y starts with an end")
pd.DataFrame(test)[pd.DataFrame(test)["trip"]==False]

Entries where Bike X ends with an end and bike Y starts with an end


,trip


In [12]:
df_starts = df_4[df_4["trip"]=="start"].reset_index(drop=True)
df_ends = df_4[df_4["trip"]=="end"].reset_index(drop=True)



In [13]:
print(df_starts.head()[["b_number", "datetime", "trip"]], "\n")
print(df_ends.head()[["b_number", "datetime", "trip"]])

   b_number             datetime   trip
0     11225  2019-04-29 06:31:00  start
1     11225  2019-04-29 11:27:00  start
2     11225  2019-04-29 11:58:00  start
3     11225  2019-04-29 12:22:00  start
4     11225  2019-04-29 13:00:00  start 

   b_number             datetime trip
0     11225  2019-04-29 06:35:00  end
1     11225  2019-04-29 11:33:00  end
2     11225  2019-04-29 12:03:00  end
3     11225  2019-04-29 12:39:00  end
4     11225  2019-04-29 13:14:00  end


In [14]:
df_merged = df_starts.merge(df_ends, left_on=df_starts.index, right_on=df_ends.index, suffixes=("_start", "_end"))
df_merged.drop(["key_0", 
                "valid_start_start", 
                "valid_start_end", 
                "trip_start", 
                "trip_end", 
                "b_number_end", 
                "b_boardcomputer_end", 
                "b_pedelec_battery_end", 
                "b_lock_types_end", 
                "b_bike_type_end", 
                "b_electric_lock_end"], axis=1, inplace=True)
df_merged.head()

,p_spot_start,b_lock_types_start,p_maintenance_start,p_bike_racks_start,p_terminal_type_start,p_place_type_start,p_number_start,p_uid_start,b_number_start,p_free_racks_start,...,p_uid_end,p_free_racks_end,datetime_end,p_lng_end,p_address_end,p_bikes_end,p_booked_bikes_end,p_name_end,p_lat_end,p_bike_types_end
0,True,fork_lock,False,0,NaN,0,4389.0,264595,11225,0,...,264595,0,2019-04-29 06:35:00,13.769281,NaN,1,0,Gutenbergstraße,51.058771,"{""15"": 1}"
1,False,fork_lock,False,0,NaN,12,0.0,15109859,11225,0,...,15117356,0,2019-04-29 11:33:00,8.770495,NaN,1,0,BIKE 11225,50.813655,"{""15"": 1}"
2,False,fork_lock,False,0,NaN,12,0.0,15117356,11225,0,...,15118389,0,2019-04-29 12:03:00,8.772863,NaN,1,0,BIKE 11225,50.808976,"{""15"": 1}"
3,False,fork_lock,False,0,NaN,12,0.0,15118389,11225,0,...,15119670,0,2019-04-29 12:39:00,8.774550,NaN,1,0,BIKE 11225,50.822621,"{""15"": 1}"
4,False,fork_lock,False,0,NaN,12,0.0,15119670,11225,0,...,15120946,0,2019-04-29 13:14:00,8.763238,NaN,1,0,BIKE 11225,50.813241,"{""15"": 1}"


## Merge Trips with Weather data

In [15]:
df_dwd=pd.read_csv(os.path.join(path_output_weather, "DWD.csv"))
df_dwd['MESS_DATUM'] = pd.to_datetime(df_dwd['MESS_DATUM'].astype(str), format="%Y-%m-%d %H:%M:%S", errors='coerce')

df_merged["datetime_start"] = pd.to_datetime(df_merged["datetime_start"].astype(str), format="%Y-%m-%d %H:%M:%S", errors="coerce")

In [16]:
df_full = pd.merge_asof(df_merged.sort_values('datetime_start'), 
                        df_dwd, left_on='datetime_start', 
                        right_on='MESS_DATUM', 
                        tolerance=pd.Timedelta('30 min'),
                        allow_exact_matches=True, direction='nearest')

In [17]:
df_full.head(5)

,p_spot_start,b_lock_types_start,p_maintenance_start,p_bike_racks_start,p_terminal_type_start,p_place_type_start,p_number_start,p_uid_start,b_number_start,p_free_racks_start,...,p_lat_end,p_bike_types_end,Unnamed: 0,MESS_DATUM,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms
0,False,analog_code_lock,False,0,NaN,12,0.0,12095573,93771,0,...,51.046234,"{""15"": 1}",1752,2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
1,True,analog_code_lock,False,0,free,0,4486.0,10299640,93576,0,...,51.049069,"{""15"": 1}",1752,2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
2,True,frame_lock,False,0,free,0,4483.0,10299584,93440,0,...,51.028370,"{""undefined"": 1, ""15"": 2}",1752,2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
3,True,analog_code_lock,False,0,NaN,0,4373.0,264575,93585,0,...,51.071740,"{""undefined"": 3, ""15"": 2}",1752,2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
4,False,analog_code_lock,False,0,NaN,12,0.0,12098234,93322,0,...,51.041798,"{""undefined"": 1}",1752,2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3


In [19]:
df_full.drop("Unnamed: 0", inplace=True, axis=1)

KeyError: "['Unnamed: 0'] not found in axis"

In [20]:
# cast start&end times to datetimes
df_full['datetime_start'] = pd.to_datetime(df_full['datetime_start'])
df_full['datetime_end'] = pd.to_datetime(df_full['datetime_end'])
# calculate trip duration
df_full['trip_duration'] = ((df_full['datetime_end'] - df_full['datetime_start']).dt.total_seconds() / 60.0).round(2)

In [23]:
df_full.to_csv(os.path.join(path_output, "Trips.csv"))

In [24]:
df_full.head()

,p_spot_start,b_lock_types_start,p_maintenance_start,p_bike_racks_start,p_terminal_type_start,p_place_type_start,p_number_start,p_uid_start,b_number_start,p_free_racks_start,...,p_lat_end,p_bike_types_end,MESS_DATUM,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms,trip_duration
0,False,analog_code_lock,False,0,NaN,12,0.0,12095573,93771,0,...,51.046234,"{""15"": 1}",2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3,28.0
1,True,analog_code_lock,False,0,free,0,4486.0,10299640,93576,0,...,51.049069,"{""15"": 1}",2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3,10.0
2,True,frame_lock,False,0,free,0,4483.0,10299584,93440,0,...,51.028370,"{""undefined"": 1, ""15"": 2}",2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3,26.0
3,True,analog_code_lock,False,0,NaN,0,4373.0,264575,93585,0,...,51.071740,"{""undefined"": 3, ""15"": 2}",2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3,28.0
4,False,analog_code_lock,False,0,NaN,12,0.0,12098234,93322,0,...,51.041798,"{""undefined"": 1}",2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3,2.0
